In [1]:
!pip install bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 20.4 MB/s eta 0:00:00:00:0100:01


In [ ]:
import os
from huggingface_hub import login
import datasets
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re

In [3]:
os.environ["HUGGINGFACE_TOKEN"] = "your_token"
login(os.environ["HUGGINGFACE_TOKEN"])

In [1]:
dataset = "chengxuphd/liar2"
dataset = datasets.load_dataset(dataset)

README.md:   0%|          | 0.00/2.87k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18369 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2297 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2296 [00:00<?, ? examples/s]

In [ ]:
# Convert labels: {0, 1, 2, 3} -> 1, {4, 5} -> 0
def convert_labels(data):
    if data["label"] in [0, 1, 2, 3]:
        data["label"] = 1
    else:
        data["label"] = 0
    return data

In [3]:
dataset = dataset.map(convert_labels)
test_dataset = dataset["test"]

Map:   0%|          | 0/18369 [00:00<?, ? examples/s]

Map:   0%|          | 0/2297 [00:00<?, ? examples/s]

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

In [9]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [33]:
# Zero-shot prompt template
def format_chat_prompt_v1(claim):
    """Formats input using the chat template for better responses (zero-shot)."""
    user_input = f"""Classify the following claim as FAKE or REAL, and highlight only the top five keywords contributing to the classification.
    
    Focus only on linguistic features (tone, exaggeration, emotional appeal) and do NOT use any factual knowledge.
    
    Claim: "{claim}"
    
    Output the result in the following structured format:
    
    Classification: (FAKE or REAL)
    
    Keywords: [list of ONLY five important words]
    
    Linguistic Reasoning: (Explain based on language features such as exaggeration, strong emotions, vagueness, etc.)"""

    chat_prompt = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are an AI that detects fake news using linguistic analysis, focusing on tone and word choice, without relying on factual knowledge."},
            {"role": "user", "content": user_input}
        ],
        tokenize=False,  # Keep raw string for LLM
        add_generation_prompt=True,  # Adds assistant's turn automatically
    )
    return chat_prompt

# Few-shot prompt template
def format_chat_prompt_v2(claim):
    """Formats input using the chat template for better responses with few-shot examples."""

    chat_prompt = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are an AI that classifies a piece of text as FAKE or REAL, using linguistic analysis, focusing on tone and word choice, without relying on factual knowledge."},
            
            # Example 1 (FAKE)
            {"role": "user", "content": """Classify the following claim as FAKE or REAL, and highlight only the top five keywords contributing to the classification.

            Do NOT give any classification other than FAKE or REAL.
            
            Focus only on linguistic features (tone, exaggeration, emotional appeal) and do NOT use any factual knowledge.
            
            Claim: "President Obama used government funds to pay for his personal vacations for the next 20 years and Trump sent him a bill."
            
            Output the result in the following structured format:
    
            Classification: (FAKE or REAL)
    
            Keywords: [list of ONLY five important words]
    
            Linguistic Reasoning: (Explain based on language features such as exaggeration, strong emotions, vagueness, etc.)"""},
            
            {"role": "assistant", "content": """Classification: FAKE
            
            Keywords: "government funds", "personal vacations", "20 years", "Trump", "bill"
            
            Linguistic Reasoning: The claim uses exaggeration, sensational language, and vague accusations (e.g., "20 years", "government funds") to create drama and suspicion without evidence."""},

            # Example 2 (REAL)
            {"role": "user", "content": """Classify the following claim as FAKE or REAL, and highlight only the top five keywords contributing to the classification.
            
            Do NOT give any classification other than FAKE or REAL.
            
            Focus only on linguistic features (tone, exaggeration, emotional appeal) and do NOT use any factual knowledge.
            
            Claim: "Despite having their budget increased by over 40 percent since 2009, pending claims for benefits with the Department of Veterans Affairs have increased from 391,000 to 890,000 under the Obama Administration."
            
            Output the result in the following structured format:
    
            Classification: (FAKE or REAL)
    
            Keywords: [list of ONLY five important words]
    
            Linguistic Reasoning: (Explain based on language features such as exaggeration, strong emotions, vagueness, etc.)"""},
            
            {"role": "assistant", "content": """Classification: REAL
            
            Keywords: "budget", "40 percent", "pending claims", "Department of Veterans Affairs", "Obama Administration"
            
            Linguistic Reasoning: The claim presents specific figures, neutral wording, and clear attribution, making it sound objective and fact-based rather than emotional or exaggerated."""},

            {"role": "user", "content": f"""Classify the following claim as FAKE or REAL, and highlight only the top five keywords contributing to the classification.
            
            Do NOT give any classification other than FAKE or REAL.
            
            Focus only on linguistic features (tone, exaggeration, emotional appeal) and do NOT use any factual knowledge.
            
            Claim: "{claim}"
            
            Output the result in the following structured format:
    
            Classification: (FAKE or REAL)
    
            Keywords: [list of ONLY five important words]
    
            Linguistic Reasoning: (Explain based on language features such as exaggeration, strong emotions, vagueness, etc.)"""}
        ],
        tokenize=False,  # Keep raw string for LLM
        add_generation_prompt=True,  # Adds assistant's turn automatically
    )
    
    return chat_prompt

In [11]:
# Extract classification, keywords, and reasoning from model output
def extract_prediction(response):
    # Extract classification (FAKE or REAL)
    classification_match = re.search(r'Classification:\s*(FAKE|REAL)', response)
    classification = classification_match.group(1) if classification_match else "UNKNOWN"

    # Extract keywords as a list
    keywords_match = re.search(r'Keywords:\s*(.*)', response)
    
    if keywords_match:
        keywords_text = keywords_match.group(1).strip()
        
        # Split by commas and remove any stray quotes/spaces
        keywords = [kw.strip().strip('"').strip("'") for kw in keywords_text.split(",")]
    else:
        keywords = []

    # Extract linguistic reasoning
    reasoning_match = re.search(r'Linguistic Reasoning:\s*(.*)', response, re.DOTALL)
    reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided."
    
    return classification, keywords, reasoning

In [12]:
# Perform evaluation to extract results from test dataset into a .csv file
def evaluate(shot=0):
    results = []
    unknown_count = 0
    
    for sample in test_dataset:
        claim = sample["statement"]
        true_label = sample["label"]
    
        # Format prompt using chat template
        if prompt_type == 0:
            formatted_prompt = format_chat_prompt_v1(claim)
        else:
            formatted_prompt = format_chat_prompt_v2(claim)
    
        # Tokenize and generate response
        inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
        output_tokens = model.generate(**inputs, max_new_tokens=150)
        model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
        cleaned_response = model_response.split("[/INST]")[-1].strip()
    
        # Extract structured outputs
        predicted_label, keywords, reasoning = extract_prediction(cleaned_response)
    
        # Convert predicted label to binary (FAKE=1, REAL=0)
        if predicted_label == "FAKE":
            pred_label = 1
        elif predicted_label == "REAL":
            pred_label = 0
        else: # Unknown label predicted
            print(predicted_label)
            pred_label = -1
            unknown_count += 1
        
        results.append({
                "Claim": claim,
                "Prediction": pred_label,
                "Keywords": ", ".join(keywords),
                "Linguistic Reasoning": reasoning,
                "True Label": true_label
            })
    
    df = pd.DataFrame(results)
    df.to_csv("results_zeroshot.csv", index=False)
    print(f"Number of test instances where model outputs an unknown classification: {unknown_count}")
    print(f"% of test instances where model outputs either FAKE or REAL: {(len(test_dataset) - unknown_count) / len(test_dataset)*100:.2f}")

In [ ]:
evaluate(shot=0) # zero-shot

In [ ]:
# Evaluation Metrics

df = pd.read_csv('results_zeroshot.csv')

# Filter out rows with Prediction == -1
df = df[df["Prediction"] != -1]

df_class_0 = df[df["True Label"] == 0]
df_class_1 = df[df["True Label"] == 1]

minority_class_size = min(len(df_class_0), len(df_class_1))

metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': []
}

# Run the sampling and evaluation 5 times
for _ in range(5):
    if len(df_class_0) < len(df_class_1):
        sampled_majority = df_class_1.sample(n=minority_class_size, random_state=np.random.randint(0, 10000))
        balanced_df = pd.concat([df_class_0, sampled_majority])
    else:
        sampled_majority = df_class_0.sample(n=minority_class_size, random_state=np.random.randint(0, 10000))
        balanced_df = pd.concat([df_class_1, sampled_majority])

    # Shuffle the balanced dataset
    balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

    y_true = balanced_df["True Label"]
    y_pred = balanced_df["Prediction"]

    # Compute metrics
    metrics['accuracy'].append(accuracy_score(y_true, y_pred))
    metrics['precision'].append(precision_score(y_true, y_pred))
    metrics['recall'].append(recall_score(y_true, y_pred))
    metrics['f1'].append(f1_score(y_true, y_pred))

# Compute and print average metrics
avg_metrics = {k: np.mean(v) for k, v in metrics.items()}
print("Averaged Metrics over 5 runs:")
for metric, score in avg_metrics.items():
    print(f"{metric.capitalize()}: {score:.2f}")

## Debugging Purposes

In [24]:
# for i in range(10):
#     train_claim = dataset["train"]["statement"][i]
#     train_label = dataset["train"]["label"][i]
#     print(f"Claim:\n{train_claim}")
#     formatted_prompt = format_chat_prompt_v1(train_claim)
    
#     inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
#     output_tokens = model.generate(**inputs, max_new_tokens=150)
#     model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
#     cleaned_response = model_response.split("[/INST]")[-1].strip()
#     print(f"Model Response:\n{cleaned_response}")
#     print(f"True Label:\n{train_label}")

#     print(extract_prediction(cleaned_response))
#     print("\n#########################")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Claim:
90 percent of Americans "support universal background checks" for gun purchases.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Response:
Classification: FAKE

Keywords: "90 percent of Americans", "support", "universal background checks"

Linguistic Reasoning: The use of the quantifier "90 percent" is an exaggeration, as it is unlikely that such a high percentage of Americans would support a single policy position. The word "support" is vague and can be interpreted in different ways. The term "universal background checks" is emotionally charged and can evoke strong reactions from different groups, making the statement more likely to be a subject of debate rather than a fact.
True Label:
1
('FAKE', ['90 percent of Americans', 'support', 'universal background checks'], 'The use of the quantifier "90 percent" is an exaggeration, as it is unlikely that such a high percentage of Americans would support a single policy position. The word "support" is vague and can be interpreted in different ways. The term "universal background checks" is emotionally charged and can evoke strong reactions from different groups,

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Model Response:
Classification: REAL

Keywords: Jeff Reardon, cut, elementary school, music classes, art instruction

Linguistic Reasoning: The claim uses specific names and actions, which adds to its credibility. The use of the verb "cut" implies a clear and definitive action, and the mention of specific programs such as music classes and art instruction further supports the claim's authenticity. The tone of the claim is factual and straightforward, with no obvious exaggeration or emotional appeal.
True Label:
0
('REAL', ['Jeff Reardon', 'cut', 'elementary school', 'music classes', 'art instruction'], 'The claim uses specific names and actions, which adds to its credibility. The use of the verb "cut" implies a clear and definitive action, and the mention of specific programs such as music classes and art instruction further supports the claim\'s authenticity. The tone of the claim is factual and straightforward, with no obvious exaggeration or emotional appeal.')

####################

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Response:
Classification: FAKE

Keywords: one in three, women, dangerous journey, north, sexually assaulted

Linguistic Reasoning: The statement "One in three women is sexually assaulted on the dangerous journey north" is an exaggeration. The use of the phrase "one in three" is a statistical claim that implies a high prevalence, which is likely to evoke strong emotions and concern. However, without specific facts or data, this claim is vague and unverifiable. The words "dangerous journey north" also contribute to the emotional appeal and sense of danger, further increasing the likelihood of the statement being perceived as sensational or exaggerated.
True Label:
1
('FAKE', ['one in three', 'women', 'dangerous journey', 'north', 'sexually assaulted'], 'The statement "One in three women is sexually assaulted on the dangerous journey north" is an exaggeration. The use of the phrase "one in three" is a statistical claim that implies a high prevalence, which is likely to evoke strong 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Response:
Classification: REAL

Keywords: it, actually, safest, time, fly

Linguistic Reasoning: The claim is stated in a matter-of-fact tone with no clear signs of exaggeration or emotional appeal. The use of the words "it's actually" implies a correction or contradiction of a common belief, which could be an attempt to downplay the risk of flying. However, without additional context, it's important to note that this classification is based solely on linguistic features and does not take into account any factual information or the specific meaning of the word "safest."
True Label:
0
('REAL', ['it', 'actually', 'safest', 'time', 'fly'], 'The claim is stated in a matter-of-fact tone with no clear signs of exaggeration or emotional appeal. The use of the words "it\'s actually" implies a correction or contradiction of a common belief, which could be an attempt to downplay the risk of flying. However, without additional context, it\'s important to note that this classification is bas

In [34]:
# for i in range(10):
#     train_claim = dataset["train"]["statement"][i]
#     train_label = dataset["train"]["label"][i]
#     print(f"Claim:\n{train_claim}")
#     formatted_prompt = format_chat_prompt_v2(train_claim)
    
#     inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
#     output_tokens = model.generate(**inputs, max_new_tokens=150)
#     model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
#     cleaned_response = model_response.split("[/INST]")[-1].strip()
#     print(f"Model Response:\n{cleaned_response}")
#     print(f"True Label:\n{train_label}")
    
#     print(extract_prediction(cleaned_response))
#     print("\n#########################")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Claim:
90 percent of Americans "support universal background checks" for gun purchases.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Response:
Classification: FAKE

Keywords: "90 percent", "Americans", "support", "universal background checks", "gun purchases"

Linguistic Reasoning: The claim uses a high percentage figure to create a strong emotional appeal and exaggeration, which is not supported by any factual evidence.
True Label:
0
('FAKE', ['90 percent', 'Americans', 'support', 'universal background checks', 'gun purchases'], 'The claim uses a high percentage figure to create a strong emotional appeal and exaggeration, which is not supported by any factual evidence.')

#########################
Claim:
Last year was one of the deadliest years ever for law enforcement officers.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Model Response:
Classification: FAKE
            
            Keywords: "Says", "Jeff Reardon", "cut", "elementary school", "music classes"
            
            Linguistic Reasoning: The claim starts with "Says", which is a vague and ambiguous way to introduce information, and then lists several unspecific actions attributed to "Jeff Reardon" without providing any context or evidence. This lack of specificity and the use of vague language make the claim less credible.
True Label:
1
('FAKE', ['Says', 'Jeff Reardon', 'cut', 'elementary school', 'music classes'], 'The claim starts with "Says", which is a vague and ambiguous way to introduce information, and then lists several unspecific actions attributed to "Jeff Reardon" without providing any context or evidence. This lack of specificity and the use of vague language make the claim less credible.')

#########################
Claim:
One in three women is sexually assaulted on the dangerous journey north.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Response:
Classification: FAKE

Keywords: "one in three women", "sexually assaulted", "dangerous journey north"

Linguistic Reasoning: The claim uses an exaggerated statistic and strong emotional language to create a sense of urgency and fear, but it lacks specificity and concrete evidence.
True Label:
0
('FAKE', ['one in three women', 'sexually assaulted', 'dangerous journey north'], 'The claim uses an exaggerated statistic and strong emotional language to create a sense of urgency and fear, but it lacks specificity and concrete evidence.')

#########################
Claim:
It's actually the safest time to fly.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Response:
Classification: NEUTRAL

Keywords: "safest time", "fly"

Linguistic Reasoning: The claim is vague and lacks clear exaggeration or emotional appeal. It is a subjective statement that can be argued both ways, making it neutral.
True Label:
1
('UNKNOWN', ['safest time', 'fly'], 'The claim is vague and lacks clear exaggeration or emotional appeal. It is a subjective statement that can be argued both ways, making it neutral.')

#########################
Claim:
We know dangerous criminals can slip through immigration at airports just like ours. Yet Joe Donnelly stood right here with the radical left against all efforts to secure our most vulnerable entry points.
Model Response:
Classification: FAKE

Keywords: "dangerous criminals", "slip through immigration", "airports", "Joe Donnelly", "radical left"

Linguistic Reasoning: The claim uses strong emotional language ("dangerous criminals"), vague accusations ("can slip through"), and polarizing terms ("radical left") to create 